In [2]:
import pandas as pd
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('firstProject').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/08 11:58:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df = spark.read.csv('Google-Playstore.csv',header=True,inferSchema=True)

In [5]:
df.count()

2312944

In [6]:
df.printSchema()

root
 |-- App Name: string (nullable = true)
 |-- App Id: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Rating Count: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Minimum Installs: string (nullable = true)
 |-- Maximum Installs: string (nullable = true)
 |-- Free: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Currency: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Minimum Android: string (nullable = true)
 |-- Developer Id: string (nullable = true)
 |-- Developer Website: string (nullable = true)
 |-- Developer Email: string (nullable = true)
 |-- Released: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Privacy Policy: string (nullable = true)
 |-- Ad Supported: string (nullable = true)
 |-- In App Purchases: string (nullable = true)
 |-- Editors Choice: string (nullable = true)
 |-- Scra

# Data Cleaning 
1. Removing non-required column

In [7]:
df.show(1)

+--------+-------------------+---------+------+------------+--------+----------------+----------------+----+-----+--------+----+---------------+--------------------+--------------------+--------------------+------------+------------+--------------+--------------------+------------+----------------+--------------+-------------------+
|App Name|             App Id| Category|Rating|Rating Count|Installs|Minimum Installs|Maximum Installs|Free|Price|Currency|Size|Minimum Android|        Developer Id|   Developer Website|     Developer Email|    Released|Last Updated|Content Rating|      Privacy Policy|Ad Supported|In App Purchases|Editors Choice|       Scraped Time|
+--------+-------------------+---------+------+------------+--------+----------------+----------------+----+-----+--------+----+---------------+--------------------+--------------------+--------------------+------------+------------+--------------+--------------------+------------+----------------+--------------+----------------

In [8]:
df = df.drop('App Id','Rating Count','Minimum Installs','Maximum Installs','Minimum Android','Developer Id','Developer Website')

In [9]:
df = df.drop('Developer Email','Released','Last Updated','Privacy Policy','Ad Supported','In App Purchases','Editors Choice','Scraped Time')

In [10]:
df.show(5)

+--------------------+-------------+------+--------+----+-----+--------+----+--------------+
|            App Name|     Category|Rating|Installs|Free|Price|Currency|Size|Content Rating|
+--------------------+-------------+------+--------+----+-----+--------+----+--------------+
|             Gakondo|    Adventure|   0.0|     10+|True|    0|     USD| 10M|      Everyone|
| Ampere Battery Info|        Tools|   4.4|  5,000+|True|    0|     USD|2.9M|      Everyone|
|              Vibook| Productivity|   0.0|     50+|True|    0|     USD|3.7M|      Everyone|
|Smart City Trichy...|Communication|   5.0|     10+|True|    0|     USD|1.8M|      Everyone|
|             GROW.me|        Tools|   0.0|    100+|True|    0|     USD|6.2M|      Everyone|
+--------------------+-------------+------+--------+----+-----+--------+----+--------------+
only showing top 5 rows



# Removing unwanted character and changing datatype to proper datatype

In [11]:
from pyspark.sql.functions import regexp_replace, col
from pyspark.sql.types import IntegerType, DoubleType, BooleanType

In [12]:
df = df.withColumn('Installs',regexp_replace('Installs','[^0-9]',''))

In [13]:
df.show(5)

+--------------------+-------------+------+--------+----+-----+--------+----+--------------+
|            App Name|     Category|Rating|Installs|Free|Price|Currency|Size|Content Rating|
+--------------------+-------------+------+--------+----+-----+--------+----+--------------+
|             Gakondo|    Adventure|   0.0|      10|True|    0|     USD| 10M|      Everyone|
| Ampere Battery Info|        Tools|   4.4|    5000|True|    0|     USD|2.9M|      Everyone|
|              Vibook| Productivity|   0.0|      50|True|    0|     USD|3.7M|      Everyone|
|Smart City Trichy...|Communication|   5.0|      10|True|    0|     USD|1.8M|      Everyone|
|             GROW.me|        Tools|   0.0|     100|True|    0|     USD|6.2M|      Everyone|
+--------------------+-------------+------+--------+----+-----+--------+----+--------------+
only showing top 5 rows



In [14]:
df.printSchema()

root
 |-- App Name: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Free: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Currency: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Content Rating: string (nullable = true)



changing dataType

In [15]:
df = df.withColumn('Installs',col('Installs').cast('int'))\
             .withColumn('Rating',col('Rating').cast('double'))\
             .withColumn('Free',col('Free').cast('boolean'))\
             .withColumn('Price',col('Price').cast('double'))

In [16]:
df.show(5)

+--------------------+-------------+------+--------+----+-----+--------+----+--------------+
|            App Name|     Category|Rating|Installs|Free|Price|Currency|Size|Content Rating|
+--------------------+-------------+------+--------+----+-----+--------+----+--------------+
|             Gakondo|    Adventure|   0.0|      10|true|  0.0|     USD| 10M|      Everyone|
| Ampere Battery Info|        Tools|   4.4|    5000|true|  0.0|     USD|2.9M|      Everyone|
|              Vibook| Productivity|   0.0|      50|true|  0.0|     USD|3.7M|      Everyone|
|Smart City Trichy...|Communication|   5.0|      10|true|  0.0|     USD|1.8M|      Everyone|
|             GROW.me|        Tools|   0.0|     100|true|  0.0|     USD|6.2M|      Everyone|
+--------------------+-------------+------+--------+----+-----+--------+----+--------------+
only showing top 5 rows



In [17]:
df.printSchema()

root
 |-- App Name: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Installs: integer (nullable = true)
 |-- Free: boolean (nullable = true)
 |-- Price: double (nullable = true)
 |-- Currency: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Content Rating: string (nullable = true)



# Performing ETL 

Renamimg column and checking count of duplicate values first

In [18]:
df = df.withColumnRenamed('App Name','AppName')

In [19]:
df.createOrReplaceTempView('app')

In [20]:
spark.sql('select AppName, count(1) from app \
            group by AppName order by count(1) desc limit 10;').show()

+--------------+--------+
|       AppName|count(1)|
+--------------+--------+
|   Tic Tac Toe|     382|
|    Calculator|     261|
|    Flashlight|     256|
|BMI Calculator|     201|
|Age Calculator|     190|
|        Sudoku|     174|
|       Gallery|     164|
|Unit Converter|     151|
|  Music Player|     147|
|     Solitaire|     145|
+--------------+--------+



# select top 10 install apps

In [21]:
spark.sql('select AppName, sum(Installs) as total_install\
          from app \
            group by AppName order by sum(Installs) desc limit 10;').show()

+--------------------+-------------+
|             AppName|total_install|
+--------------------+-------------+
|       YouTube Music|   2010000000|
|          Calculator|   1534571379|
|              TikTok|   1100000000|
|Samsung Security ...|   1100000000|
|            Messages|   1024723601|
|               Clock|   1001061360|
|       Secure Folder|   1000250000|
|        Android Auto|   1000010000|
|         Device Care|   1000010000|
|            Briefing|   1000001050|
+--------------------+-------------+



In [22]:
df.show(3)

+-------------------+------------+------+--------+----+-----+--------+----+--------------+
|            AppName|    Category|Rating|Installs|Free|Price|Currency|Size|Content Rating|
+-------------------+------------+------+--------+----+-----+--------+----+--------------+
|            Gakondo|   Adventure|   0.0|      10|true|  0.0|     USD| 10M|      Everyone|
|Ampere Battery Info|       Tools|   4.4|    5000|true|  0.0|     USD|2.9M|      Everyone|
|             Vibook|Productivity|   0.0|      50|true|  0.0|     USD|3.7M|      Everyone|
+-------------------+------------+------+--------+----+-----+--------+----+--------------+
only showing top 3 rows



# Top 10 installed apps and type of distribution (free/paid)

In [23]:
spark.sql('select AppName, Free, sum(Installs) as total_install\
          from app \
            group by AppName, Free order by sum(Installs) desc limit 10;').show()

+--------------------+----+-------------+
|             AppName|Free|total_install|
+--------------------+----+-------------+
|       YouTube Music|true|   2010000000|
|          Calculator|true|   1534571378|
|              TikTok|true|   1100000000|
|Samsung Security ...|true|   1100000000|
|            Messages|true|   1024723601|
|               Clock|true|   1001061360|
|       Secure Folder|true|   1000250000|
|        Android Auto|true|   1000010000|
|         Device Care|true|   1000010000|
|            Briefing|true|   1000001050|
+--------------------+----+-------------+



# Category wise distribution of installed apps

In [24]:
df.show(3)

+-------------------+------------+------+--------+----+-----+--------+----+--------------+
|            AppName|    Category|Rating|Installs|Free|Price|Currency|Size|Content Rating|
+-------------------+------------+------+--------+----+-----+--------+----+--------------+
|            Gakondo|   Adventure|   0.0|      10|true|  0.0|     USD| 10M|      Everyone|
|Ampere Battery Info|       Tools|   4.4|    5000|true|  0.0|     USD|2.9M|      Everyone|
|             Vibook|Productivity|   0.0|      50|true|  0.0|     USD|3.7M|      Everyone|
+-------------------+------------+------+--------+----+-----+--------+----+--------------+
only showing top 3 rows



In [27]:
spark.sql('select Category, sum(installs) as total_installs \
         from app \
         group by Category order by sum(installs) desc limit 10;').show()

+-------------+--------------+
|     Category|total_installs|
+-------------+--------------+
|        Tools|   46440471469|
| Productivity|   23314050628|
|Communication|   18216594754|
|       Action|   17400377388|
|Entertainment|   17108498394|
|       Casual|   16837071701|
|       Arcade|   14501333037|
|Music & Audio|   14239411974|
|  Photography|   13998958963|
|       Social|   12165995316|
+-------------+--------------+



# Top Paid apps

In [28]:
df.show(3)

+-------------------+------------+------+--------+----+-----+--------+----+--------------+
|            AppName|    Category|Rating|Installs|Free|Price|Currency|Size|Content Rating|
+-------------------+------------+------+--------+----+-----+--------+----+--------------+
|            Gakondo|   Adventure|   0.0|      10|true|  0.0|     USD| 10M|      Everyone|
|Ampere Battery Info|       Tools|   4.4|    5000|true|  0.0|     USD|2.9M|      Everyone|
|             Vibook|Productivity|   0.0|      50|true|  0.0|     USD|3.7M|      Everyone|
+-------------------+------------+------+--------+----+-----+--------+----+--------------+
only showing top 3 rows



In [33]:
spark.sql('select AppName, Free, sum(Installs) as total_installs \
            from app \
            where Free = False \
         group by AppName, Free \
         order by sum(Installs) desc').show(10)

+--------------------+-----+--------------+
|             AppName| Free|total_installs|
+--------------------+-----+--------------+
|       Hitman Sniper|false|      10000000|
|           Minecraft|false|      10000000|
|Stickman Downhill...|false|      10000000|
|Stickman Legends-...|false|      10000000|
|          True Skate|false|       5000000|
|Poweramp Full Ver...|false|       5000000|
|League of Stickma...|false|       5000000|
|RFS - Real Flight...|false|       1000000|
|      Draw Something|false|       1000000|
|         Smart Tools|false|       1000000|
+--------------------+-----+--------------+
only showing top 10 rows



# Top paid rating apps

In [34]:
df.show(3)

+-------------------+------------+------+--------+----+-----+--------+----+--------------+
|            AppName|    Category|Rating|Installs|Free|Price|Currency|Size|Content Rating|
+-------------------+------------+------+--------+----+-----+--------+----+--------------+
|            Gakondo|   Adventure|   0.0|      10|true|  0.0|     USD| 10M|      Everyone|
|Ampere Battery Info|       Tools|   4.4|    5000|true|  0.0|     USD|2.9M|      Everyone|
|             Vibook|Productivity|   0.0|      50|true|  0.0|     USD|3.7M|      Everyone|
+-------------------+------------+------+--------+----+-----+--------+----+--------------+
only showing top 3 rows



In [37]:
spark.sql('select AppName, Rating \
            from app \
            order by Rating desc').show(10)

+--------------------+------+
|             AppName|Rating|
+--------------------+------+
|भारतीय पुरावा अधि...|   5.0|
|FM INTENSIDAD SAN...|   5.0|
|  GROVES Buho Market|   5.0|
|   FreeMove CrossFit|   5.0|
|               Smooa|   5.0|
|             Footbar|   5.0|
|DIGIPASS SecureCl...|   5.0|
|Juniors Kiddo - P...|   5.0|
|           SensorLab|   5.0|
|StickerPixels - S...|   5.0|
+--------------------+------+
only showing top 10 rows

